In [ ]:
import pandas as pd
from database.connect import getConnection
import matplotlib.pyplot as plt
from datetime import datetime
import numpy as np
import seaborn as sns
sns.set()
from pathlib import Path

# **CLEAR OUTPUTS BEFORE PUSHING TO GIT**
Press the clear all outputs button in the toolbar above before pushing to git.  
This makes version control easier and avoids merge conflicts. 

In [ ]:
start_date = '01-01-2000'
end_date = '01-01-2023'

# convert start and end date to unix timestamp in milliseconds
start_date_unix = int(datetime.strptime(start_date, '%d-%m-%Y').timestamp() * 1000)
end_date_unix = int(datetime.strptime(end_date, '%d-%m-%Y').timestamp() * 1000)

# create a part you can insert into your where clause
date_restriction = f" AND tweets.timestamp_ms >= {start_date_unix} AND tweets.timestamp_ms <= {end_date_unix}"

The cell below creates the connection to the database and the cursor to execute queries.  
It also contains variables relevant everywhere in the notebook.

In [ ]:
try:
    connection = getConnection()
except Exception:
    print("✖️ Error while connecting to MySQL engine database.")
    print("ℹ️ Please make sure the environment file `.env` is located at"+
        "the project root directory and contains proper configuration.")
    raise

cursor = connection.cursor()

categories = {'booking': ['booking', 'booked', 'book', 'ticket', 'tickets'],
            'canceling': ['canceled', 'cancellations'],
            'money': ['refund', 'compensation', 'claim', 'money', 'pay', 'paid'],
            'baggage': ['bag', 'baggage', 'luggage', 'bags'],
            'staff': ['staff', 'crew'],
            'waiting': ['waiting', 'delay', 'wait'],
            'boarding': ['boarding'],
            'stuck': ['stuck'],

            'information': ['info', 'information'],
            'customers': ['customer', 'customers', 'passenger', 'passengers'],
            'dm': [' dm'], 
            }

airlines_dict = {'KLM': ['klm'],
                'AirFrance':['airfrance',
                            'air france'],
                'British_Airways': ['british_airways',
                                    'british airways'],
                'AmericanAir': ['americanair',
                                'american airlines'],
                'Lufthansa': ['lufthansa'],
                'AirBerlin': ['airberlin',
                                'air berlin'],
                'AirBerlin assist': ['airberlin assist',
                                    'air berlin assist',
                                    'airberlinassist'],
                'easyJet': ['easyjet'],
                'RyanAir': ['ryanair'],
                'SingaporeAir': ['singaporeair',
                                'singapore airlines'],
                'Qantas': ['qantas'],
                'EtihadAirways': ['etihad airways',
                                'etihadairways',
                                'etihad'],
                'VirginAtlantic': ['virgin atlantic',
                                    'virginatlantic'],
            }

airlines_of_interest = ['AmericanAir', 'Other', 'British_Airways']

In [ ]:
# create a dataframe with the counts of each category
excluided = ['dm', 'customers', 'information']
relevant_categories = [key for key in categories.keys() if key not in excluided]
columns = ['total'] + relevant_categories

base_path = Path('./pre-processed/')

# create a dataframe with the counts of each category, restricted by timestamp_ms
df = pd.DataFrame(columns=columns)
for airline in airlines_of_interest:
    df_airline = pd.read_csv(base_path / f'{airline}_category_counts.csv')
    length = len(df_airline.index)
    df_airline = df_airline[df_airline['timestamp_ms'] >= start_date_unix]
    df_airline = df_airline[df_airline['timestamp_ms'] <= end_date_unix]
    df_airline = df_airline.sum().to_frame().T
    df_airline['total'] = length
    df_airline['airline'] = airline
    df = pd.concat([df, df_airline])


df = df.set_index('airline')
df


In [ ]:
# turn the dataframe into a table that shows both the counts, aswell as the keywords
df_category_counts_table = df_category_counts.copy()
df_category_counts_table = df_category_counts_table.drop(columns=['total'])
df_category_counts_table = df_category_counts_table.transpose()
df_category_counts_table = df_category_counts_table.reset_index()
df_category_counts_table = df_category_counts_table.rename(columns={'index': 'category'})
df_category_counts_table = df_category_counts_table.set_index('category')

print(df_category_counts_table.head())

# add a column that displays the keywords for each category
for category in relevant_categories:
    keywords = categories[category]
    df_category_counts_table.loc[category, 'keywords'] = ", ".join(keywords)

df_category_counts_table

In [ ]:
# create a dataframe for the percentages
df_category_percentages = pd.DataFrame(columns=relevant_categories)

# create rows for each airline and initialize with NaN values
for airline in airlines_of_interest:
    df_category_percentages.loc[airline] = np.nan


# calculate the percentage of each category, restricted to 2 decimal points
for category in relevant_categories:
    if category == 'total':
        continue
    for airline in airlines_of_interest:
        raw_percentage = df_category_counts.loc[airline][category] / df_category_counts.loc[airline]['total'] * 100
        percentage = round(raw_percentage, 2)
        df_category_percentages.loc[airline, category] = percentage

# Convert relevant columns to numeric type
df_category_percentages[relevant_categories] = df_category_percentages[relevant_categories].apply(pd.to_numeric)


# create a grouped bar chart 
# df_category_percentages.plot.bar(figsize=(20, 10), fontsize=20)
# plt.title('Percentage of tweets per category', fontsize=30)
# plt.xlabel('Airline', fontsize=20)
# plt.ylabel('Percentage', fontsize=20)
# plt.legend(fontsize=20)
# plt.show()

# create a grouped bar chart grouped by category
df_category_percentages.transpose().plot.bar(figsize=(20, 10), fontsize=20)
plt.title('Percentage of tweets per category per airline', fontsize=30)
plt.xlabel('Category', fontsize=20)
plt.ylabel('Percentage', fontsize=20)
plt.legend(fontsize=20)
plt.show()